# PSS

PSS(Parameter-Space Saliency)は，Deep Learningモデルの顕著性(Saliency)を可視化する手法の一つである．

誤分類に影響したパラメータ(Weight)を分析するアプローチで，影響の大きいパラメータを補正することでDeep Learningモデルの性能を改善できることが示された．

## 参考文献

* 論文
  * https://arxiv.org/abs/2108.01335
* GitHub
  * https://github.com/LevinRoman/parameter-space-saliency
* 解説資料：DL輪読会
  * https://www.slideshare.net/DeepLearningJP2016/dlwhere-do-models-go-wrong-parameterspace-saliency-maps-for-explainability

## PSS効果例

論文ではGrad-CAMとの比較が示されている．  
下図のようにPSSではGrad-CAMでは表現されない可視化要因を表現することができる(右2枚，ユキヒメドリ(junco)及び旅客列車(passenger car))．

![paper figure19](./figure/PSS/paper_figure19.png)

## PSS理論解説

[PSS論文](https://arxiv.org/abs/2108.01335)では，パラメータ顕著性(Parameter saliency)の計算方法の説明(2.1 Parameter saliency profile)とモデルの誤動作を入力空間へ可視化する方法の説明(2.2 Input-space saliency for visualizing how filters malfunction)の2部構成で述べられる．

### Parameter saliency profile

パラメータ顕著性は下記の3ステップで計算する．

1. パラメータ毎の顕著性の計算
1. フィルタ毎の顕著性への集約
1. Validationデータによる標準化


#### パラメータ毎の顕著性の計算

入力$x$，正解ラベル$y$のValidationデータセット$D$，及び，損失関数$\mathcal{L}$で最小化したパラメータ$\theta$を持つ識別モデルを仮定する．

パラメータ毎の顕著性は，損失関数を識別モデルの各パラメータで偏微分して得られる勾配の大きさで定義する．  
インデックス$i$のパラメータを$\theta_i$で表すと，パラメータ毎の顕著性$s(x, y)_i$は以下のように定義される．

$$
  \begin{align}
    s(x, y)_i &:= |\nabla_{\theta_i}\mathcal{L}_\theta (x, y)|
  \end{align}
$$


#### フィルタ毎の顕著性への集約 

畳み込みフィルタはエッジ(Edge)，形状(Shape)，質感(Texture)を検出する性質があることで知られている．

顕著性$s(x, y)_i$をフィルタ毎に集約することにより，損失が最も敏感なフィルタを分離することが可能となる．つまり，分離されたフィルタを修正することによって，損失をより大きく減少させることが期待できる．

識別モデルの一つのフィルタを$\mathcal{F}_k$，フィルタ$\mathcal{F}_k$に属するパラメータのインデックス群(例えば，フィルタサイズが$3x3$の場合は，要素数9のインデックスベクトル)を$\alpha_k$で示す．フィルタ毎の顕著性$\bar{s}(x, y)_k$は，パラメータ毎の顕著性をフィルタ単位で平均を求めるものとして，下記のように定義される．

$$
  \begin{align}
    \bar{s}(x, y)_k &:= \frac{1}{|\alpha_k|}\sum_{i \in \alpha_k}s(x, y)_i
  \end{align}
$$



#### Validationデータによる標準化

下図(論文Figure1)の上図は，ResNet-50で層毎の顕著性を，ImageNetのValidationデータセットに対して平均値を算出し，層毎に顕著性降順にソートしたグラフである．

![paper figure1](./figure/PSS/paper_figure1.png)

勾配のスケールが入力層から出力層の間で異なっていることが明らかである(入力層の顕著性が大きく，出力層に向かうにつれて小さくなる)．これにはいくつかの要因がある．

1. 入力層に近いフィルタは，エッジ(Edge)や質感(Texture)等，幅広い画像に対して有効な特徴量を抽出する性質を持つ．
  * つまり，タスクに特化したフィルタではない為，出力層のフィルタと比較した際に相対的に損失が大きくなる
1. 一般的にネットワークを構成する際は入力層に近いほどフィルタ数が少なくなるように設計する．層あたりのフィルタ数が少ないと，各フィルタが及ぼす影響力が相対的に大きくなる．
1. 入力層に近いフィルタの効果は，後続のネットワークへ継承される．
  * つまり，出力層に向かうにつれて入力層側のフィルタで獲得した特徴量を破壊しないように影響度が小さくなる
  
そこで，スケールをフィルタ間で合わせるために，フィルタ毎にValidationデータセットで標準化する．フィルタ$k$の標準化顕著性$\hat{s}(x, y)_k$は下記のように定義される．

$$
  \begin{align}
    \hat{s}(x, y)_k &:= \frac{|\bar{s}(x, y)_k - \mu_k|}{\sigma_k}
  \end{align}
$$

これを一般化すると，

$$
  \begin{align}
    \hat{s}(x, y) &:= \frac{|\bar{s}(x, y) - \mu|}{\sigma}
  \end{align}
$$

となり，畳み込みフィルタ数長のテンソルが$\hat{s}(x, y)$として得られる．


### Input-space saliency for visualizing how filters malfunction

T.B.D